<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/Exprement%201%20(output%20files)/answer_Generation_Llama3_1_8b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# stting up

In [1]:
!pip install datasets
!huggingface-cli login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.5 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens

In [2]:
import os
import torch
import json
import pandas as pd
from google.colab import files
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from IPython.display import FileLink, display
from datasets import load_dataset

#**loading the model**

In [6]:
# Load the model and tokenizer
model_id = "meta-llama/Llama-3.1-8B"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Create the text generation pipeline
pipeline = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer
)

# Test the pipeline
print(pipeline("كم عمرك؟"))


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'كم عمرك؟ 1. How old are you?\nأنا في الثامنة عشر 2. I am'}]


# helper functions

In [ ]:
torch.cuda.empty_cache()

with the likelihood:)

In [7]:
def generate_answer(question, num_samples, model, tokenizer):
    prompt = f" أجب على السؤال التالي بجملة واحدة موجزة ولكن كاملة باللغة العربية.\nQuestion: {question}\nAnswer:"

    # Tokenize the prompt
    prompt_tokens = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate responses
    outputs = model.generate(
        **prompt_tokens,
        max_length=100,
        do_sample=True,
        temperature=0.5,
        top_p=0.95,
        num_return_sequences=num_samples,
        return_dict_in_generate=True,
        output_scores=True,
    )

    # Extract generated sequences and scores
    sequences = outputs.sequences
    scores = outputs.scores

    # Process each generated sequence
    results = []
    for i, seq in enumerate(sequences):
        # Get the text
        decoded_text = tokenizer.decode(seq, skip_special_tokens=True)
        answer = decoded_text.split("Answer:")[-1].strip()

        # Calculate log probabilities (similar to reference code)
        # We'll use model.forward to get detailed probabilities
        with torch.no_grad():
            # Only consider tokens after the prompt
            prompt_length = len(prompt_tokens.input_ids[0])
            gen_tokens = seq[prompt_length-1:]  # Include one overlap token

            # Feed the sequence to the model
            target_ids = gen_tokens.clone()
            target_ids[0] = -100  # so it Dose't compute loss for the overlap token
            model_output = model(torch.reshape(gen_tokens, (1, -1)), labels=target_ids)

            # Get average negative log likelihood
            avg_neg_log_likelihood = model_output.loss

        results.append({
            'text': answer,
            'avg_neg_log_likelihood': avg_neg_log_likelihood.item(),
        })

    return results

# main function to to load questions and gerate answers and save them to a file

In [8]:
# Main function
def main():

    # Step 1: LOAD DATA
    file_path = '/content/train-open.json'

    if os.path.exists(file_path):

        print("File found")
        with open(file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
    else:

      # Load the XQuAD dataset for Arabic (ar)
      xquad_dataset = load_dataset("xquad", "xquad.ar")
      # Load the validation set (the only available split)
      test_set = xquad_dataset['validation']

     # Create an empty list to store the data
      data = []
      # Loop through  and collect data
      for i in range(len(test_set)):  # Loop through all samples
          context = test_set[i]['context']
          question = test_set[i]['question']
          answer = test_set[i]['answers']
          # Append the data as a dictionary to the list
          data.append({
                  "Context": context,
                  "Question": question,
                  "Answer": answer
            })
    # Convert the list of dictionaries to a pandas DataFram
    df = pd.DataFrame(data)


  # Step 2: GENERATE ANSWERS
  #empty list to store the results
    results = []

    for index, row in df.iterrows():
        question = row['Question']
        original_answer = row.get('Answer', "")  # Use empty string if no original answer provided
        question_id = row.get('question_id')

        # Generate 10 answers with log probabilities for each question
        generated_results = generate_answer(question=question, num_samples=10, model=model, tokenizer=tokenizer)
        # Extract text answers and log probabilities
        generated_answers = [result['text'] for result in generated_results]
        avg_neg_log_likelihoods = [result['avg_neg_log_likelihood'] for result in generated_results]

        # Append the results
        results.append({
            "question_id": question_id,
            "question": question,
            "original_answer": original_answer,
            "generated_answers": generated_answers,
            "avg_neg_log_likelihoods": avg_neg_log_likelihoods,
        })
    # Convert the results to a DataFrame
    results_df = pd.DataFrame(results)
    print(results_df)

    #SAVE GENERATION
    filename = "generated_answers_Llama3.1-xquadALLdataset-LOG.json"

    with open(filename, "w", encoding="utf-8") as json_file:
          json.dump(results, json_file, ensure_ascii=False, indent=4)

    print("Generated answers saved to 'generated_answers.json'")
    files.download(filename)


# Run the main function
if __name__ == "__main__":
    main()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


   question_id                                           question  \
0          861  ما هي الأدلة التي وجدتها بعض الدراسات التي أجر...   
1          752                       ‏ما الهدف من هذه الاحتجاجات؟   
2          119                 ماذا كان اسما شريكي تسلا الجديدين؟   

                                     original_answer  \
0  {'text': ['فرضية منحنى كوزنتس'], 'answer_start...   
1  {'text': ['الوصول إلى النقاش'], 'answer_start'...   
2  {'text': ['روبرت لين و بنجامين فايل'], 'answer...   

                                   generated_answers  \
0  [Some studies found evidence of inequality in ...   
1  [هذه الاحتجاجات تهدف إلى إعادة إحي, ‏الاحتجاجا...   
2  [Tesla, 2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.1...   

                             avg_neg_log_likelihoods  \
0  [1.9014533758163452, 1.3061416149139404, 1.671...   
1  [1.0341473817825317, 1.6188511848449707, 1.559...   
2  [1.3906586170196533, 0.45449647307395935, 21.5...   

                           total_neg_log_likelih

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>